# Interpolation and integration algorithms

Author(s): Jukka Aho

**Abstract**: Some strategies to implement automatic differentiation. The number of different choises is caused by a fact that the linearization of function can be done before integration or vice versa, and functions can return values or do in-place modifications. There is probably performance differences between different strategies, but all of them should work.

In [1]:
using JuliaFEM
using ForwardDiff

"Old good" elasticity force equilibrium equation $R = T - F$

In [2]:
function calc_residual_vector_integrand(el::JuliaFEM.Element, xi)
    # Calculate dN/dX and interpolate material parameters
    dbasisdX = JuliaFEM.get_dbasisdX(el, xi)
    u = el.attributes["displacement"]
    lambda = JuliaFEM.interpolate(el, "lambda", xi)
    mu = JuliaFEM.interpolate(el, "mu", xi)

    # Calculate residual force vector R = T - F
    gradu = u*dbasisdX
    F = I + gradu
    E = 1/2*(gradu' + gradu + gradu'*gradu)
    S = lambda*trace(E)*I + 2*mu*E
    P = F*S
    T = P*dbasisdX'
    return T
end

calc_residual_vector_integrand (generic function with 1 method)

Test case, already well known 2d elasticity in [0,10] x [0,1] grid.

In [3]:
basis(xi) = [
    (1-xi[1])*(1-xi[2])/4
    (1+xi[1])*(1-xi[2])/4
    (1+xi[1])*(1+xi[2])/4
    (1-xi[1])*(1+xi[2])/4]
dbasis(xi) = [-(1-xi[2])/4.0    -(1-xi[1])/4.0
               (1-xi[2])/4.0    -(1+xi[1])/4.0
               (1+xi[2])/4.0     (1+xi[1])/4.0
              -(1+xi[2])/4.0     (1-xi[1])/4.0]
ipoints = 1/sqrt(3)*[-1 -1; 1 -1; 1 1; -1 1]'
iweights = [1, 1, 1, 1]
attributes = Dict()
e = JuliaFEM.Element(1, [1, 2, 3, 4], basis, dbasis, attributes, ipoints, iweights)

E = 90
nu = 0.25
mu = E/(2*(1+nu))
la = E*nu/((1+nu)*(1-2*nu))
la = 2*la*mu/(la + 2*mu)

e.attributes["coordinates"] = [0.0 0.0; 10.0 0.0; 10.0 1.0; 0.0 1.0]'
e.attributes["lambda"] = la
e.attributes["mu"] = mu
e.attributes["displacement"] = [0.0 0.0; 0.0 0.0; 0.5 0.0; 0.0 0.0]'
e.attributes["displacement nodal force"] = zeros(2, 4)
e.attributes["displacement tangent stiffness"] = zeros(8, 8);

## Integration

1. take element and function and return value
2. take function and return function which can be integrated by passing element as a function
3. do in-place integration, save values to target

In [4]:
function integrate(f::Function, el::JuliaFEM.Element)
    target = []
    for m = 1:length(el.iweights)
        w = el.iweights[m]
        xi = el.ipoints[:, m]
        J = JuliaFEM.interpolate(el, "coordinates", xi; derivative=true)
        push!(target, w*f(el, xi)*det(J))
    end
    return sum(target)
end

function integrate(f::Function)
    function integrate(el::JuliaFEM.Element)
        target = []
        for m = 1:length(el.iweights)
            w = el.iweights[m]
            xi = el.ipoints[:, m]
            J = JuliaFEM.interpolate(el, "coordinates", xi; derivative=true)
            push!(target, w*f(el, xi)*det(J))
        end
        return sum(target)
    end
    return integrate
end

function integrate!(f::Function, el::JuliaFEM.Element, target)
    # set target to zero
    el.attributes[target][:] = 0.0
    for m = 1:length(el.iweights)
        w = el.iweights[m]
        xi = el.ipoints[:, m]
        J = JuliaFEM.interpolate(el, "coordinates", xi; derivative=true)
        el.attributes[target][:,:] += w*f(el, xi)*det(J)
    end
end


integrate! (generic function with 1 method)

In [5]:
integrate(calc_residual_vector_integrand, e)

2x4 Array{Float64,2}:
 -38.2303  -72.8697  79.4912  31.6088
 -17.625   -28.475   37.7      8.4   

In [6]:
calc_residual_vector = integrate(calc_residual_vector_integrand)
calc_residual_vector(e)

2x4 Array{Float64,2}:
 -38.2303  -72.8697  79.4912  31.6088
 -17.625   -28.475   37.7      8.4   

In [7]:
integrate!(calc_residual_vector_integrand, e, "displacement nodal force")
e.attributes["displacement nodal force"]

2x4 Array{Float64,2}:
 -38.2303  -72.8697  79.4912  31.6088
 -17.625   -28.475   37.7      8.4   

## Linearization

1. take function, element and field, and return partial derivative
2. take function and field, return function which takes element as argument
3. do in-place linearization to target, requires function which takes element as argument

In general linearization can be done before integration and vice versa, i.e.

    integrate(linearize(f, "u"))(e) <-> linearize(integrate(f), "u")(e)

In [8]:
function linearize(f::Function, el::JuliaFEM.Element, field::ASCIIString)
    dim, nnodes = size(el.attributes[field])
    function helper!(x, y)
        orig = copy(el.attributes[field])
        el.attributes[field] = reshape(x, dim, nnodes)
        y[:] = f(el)
        el.attributes[field] = copy(orig)
    end
    jac = ForwardDiff.forwarddiff_jacobian(helper!, Float64, fadtype=:dual, n=dim*nnodes, m=dim*nnodes)
    return jac(el.attributes[field][:])
end

function linearize(f::Function, field::ASCIIString)
    function jacobian(el::JuliaFEM.Element, args...)
        dim, nnodes = size(el.attributes[field])
        function helper!(x, y)
            orig = copy(el.attributes[field])
            el.attributes[field] = reshape(x, dim, nnodes)
            y[:] = f(el, args...)
            el.attributes[field] = copy(orig)
        end
        jac = ForwardDiff.forwarddiff_jacobian(helper!, Float64, fadtype=:dual, n=dim*nnodes, m=dim*nnodes)
        return jac(el.attributes[field][:])
    end
    return jacobian
end

function linearize!(f::Function, el::JuliaFEM.Element, field::ASCIIString, target::ASCIIString)
    el.attributes[target][:] = 0.0
    dim, nnodes = size(el.attributes[field])
    function helper!(x, y)
        orig = copy(el.attributes[field])
        el.attributes[field] = reshape(x, dim, nnodes)
        y[:] = f(el)
        el.attributes[field] = copy(orig)
    end
    jac! = ForwardDiff.forwarddiff_jacobian!(helper!, Float64, fadtype=:dual, n=dim*nnodes, m=dim*nnodes)
    jac!(el.attributes[field][:], el.attributes[target])
end

linearize! (generic function with 1 method)

In [9]:
integrate(linearize(calc_residual_vector_integrand, "displacement"))(e)

8x8 Array{Float64,2}:
  149.721    55.55     84.679    36.65   …   -55.55   -136.278   -36.65 
   55.55    329.69     42.75    167.935     -172.941   -42.8    -324.684
   84.679    42.75    185.321   105.05      -123.05    -73.522   -24.75 
   36.65    167.935   105.05    340.54      -344.759   -24.8    -163.716
  -98.122   -55.5    -196.478  -116.9        135.8      76.233    36.6  
  -55.55   -172.941  -123.05   -344.759  …   352.922    42.8     164.778
 -136.278   -42.8     -73.522   -24.8         42.8     133.567    24.8  
  -36.65   -324.684   -24.75   -163.716      164.778    24.8     323.622

In [10]:
linearize(integrate(calc_residual_vector_integrand), "displacement")(e)

8x8 Array{Float64,2}:
  149.721    55.55     84.679    36.65   …   -55.55   -136.278   -36.65 
   55.55    329.69     42.75    167.935     -172.941   -42.8    -324.684
   84.679    42.75    185.321   105.05      -123.05    -73.522   -24.75 
   36.65    167.935   105.05    340.54      -344.759   -24.8    -163.716
  -98.122   -55.5    -196.478  -116.9        135.8      76.233    36.6  
  -55.55   -172.941  -123.05   -344.759  …   352.922    42.8     164.778
 -136.278   -42.8     -73.522   -24.8         42.8     133.567    24.8  
  -36.65   -324.684   -24.75   -163.716      164.778    24.8     323.622

In [11]:
linearize(integrate(calc_residual_vector_integrand), e, "displacement")

8x8 Array{Float64,2}:
  149.721    55.55     84.679    36.65   …   -55.55   -136.278   -36.65 
   55.55    329.69     42.75    167.935     -172.941   -42.8    -324.684
   84.679    42.75    185.321   105.05      -123.05    -73.522   -24.75 
   36.65    167.935   105.05    340.54      -344.759   -24.8    -163.716
  -98.122   -55.5    -196.478  -116.9        135.8      76.233    36.6  
  -55.55   -172.941  -123.05   -344.759  …   352.922    42.8     164.778
 -136.278   -42.8     -73.522   -24.8         42.8     133.567    24.8  
  -36.65   -324.684   -24.75   -163.716      164.778    24.8     323.622

In [12]:
linearize!(integrate(calc_residual_vector_integrand), e, "displacement", "displacement tangent stiffness")
e.attributes["displacement tangent stiffness"]

8x8 Array{Float64,2}:
  149.721    55.55     84.679    36.65   …   -55.55   -136.278   -36.65 
   55.55    329.69     42.75    167.935     -172.941   -42.8    -324.684
   84.679    42.75    185.321   105.05      -123.05    -73.522   -24.75 
   36.65    167.935   105.05    340.54      -344.759   -24.8    -163.716
  -98.122   -55.5    -196.478  -116.9        135.8      76.233    36.6  
  -55.55   -172.941  -123.05   -344.759  …   352.922    42.8     164.778
 -136.278   -42.8     -73.522   -24.8         42.8     133.567    24.8  
  -36.65   -324.684   -24.75   -163.716      164.778    24.8     323.622

In [13]:
integrate!(linearize(calc_residual_vector_integrand, "displacement"), e, "displacement tangent stiffness")
e.attributes["displacement tangent stiffness"]

8x8 Array{Float64,2}:
  149.721    55.55     84.679    36.65   …   -55.55   -136.278   -36.65 
   55.55    329.69     42.75    167.935     -172.941   -42.8    -324.684
   84.679    42.75    185.321   105.05      -123.05    -73.522   -24.75 
   36.65    167.935   105.05    340.54      -344.759   -24.8    -163.716
  -98.122   -55.5    -196.478  -116.9        135.8      76.233    36.6  
  -55.55   -172.941  -123.05   -344.759  …   352.922    42.8     164.778
 -136.278   -42.8     -73.522   -24.8         42.8     133.567    24.8  
  -36.65   -324.684   -24.75   -163.716      164.778    24.8     323.622

## Validations

In [14]:
free_dofs = [3, 4, 5, 6]
u = zeros(2, 4)
du = zeros(2, 4)
F = [0 0; 0 0; 0 -2; 0 0]'
for i=1:10
    e.attributes["displacement"] = u
    K = linearize(integrate(calc_residual_vector_integrand), "displacement")(e)
    R = integrate(calc_residual_vector_integrand)(e)
    du[free_dofs] = K[free_dofs, free_dofs] \ -(R - F)[free_dofs]
    u += du
    if norm(du) < 1.0e-9
        println("Converged in $i iterations.")
        break
    end
end
u # -2.222244754401764

Converged in 6 iterations.


2x4 Array{Float64,2}:
 0.0  -0.399145  -0.0722858  0.0
 0.0  -2.17799   -2.22224    0.0